In [1]:
import torch
import os
import random
import numpy as np
import pytorch_lightning as pl

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from dataset.utils import process_classification_dataset, load_classification_dataset

config = {
        'dataset': 'facebook_ct1', # 'infectious_ct1', #
        'hidden_size': 32,
        'rnn_layers': 8,
        'readout_layers': 3,
        'cell_type': 'lstm',
        'dim_red': 64,
        'add_self_loops': False,
        'verbose': True,
        'cat_states_layers': True,
        'weight_decay': 1e-3,
        'step_size': 30,
        'gamma': 0.5
        }

# Select one GPU if more are available
os.environ["CUDA_VISIBLE_DEVICES"]='0'

seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pl.seed_everything(seed)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Seed set to 42


In [2]:
# Fix cuda memory issue
dataset, states, node_states, node_labels = process_classification_dataset(config, "DynCRNN", device)

100%|██████████| 995/995 [11:47<00:00,  1.41it/s]


In [3]:
edge_indexes, _, _ = load_classification_dataset(config['dataset'], False)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

inputs, test_inputs, labels, test_labels = train_test_split(states.inputs[:,-1,:], dataset.targets, test_size=0.2, random_state=seed)

# Create the classifier
classifier = LogisticRegression(max_iter=5000, random_state=seed)

# Train the classifier
classifier.fit(inputs, labels)

# Test the classifier
y_pred = classifier.predict(test_inputs)

accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9547738693467337


# PCA per node

In [18]:
from torch_sparse import SparseTensor

idx = 0

edge_index = torch.cat(edge_indexes[idx], dim=1)
edge_index = torch.unique(edge_index.T, dim=0).T

num_nodes = edge_index.max().item() + 1
adj_matrix = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(num_nodes, num_nodes))
adj_matrix = adj_matrix.to_dense()

In [ ]:
from sklearn.decomposition import PCA
from einops import rearrange

node_states = rearrange(node_states[idx], 't n f -> n t f')

pca = PCA(n_components=10)
X = pca.fit_transform(rearrange(node_states, 'n t f -> (n t) f'))
X = rearrange(X, '(n t) f -> n t f', n=num_nodes, t=node_states.shape[1], f=10)
X = torch.from_numpy(X)

In [30]:
X.shape

(100, 106, 10)

In [57]:
torch.nonzero(adj_matrix).size(0)

210

In [68]:
from DMD.sindy import sindy_library_adj

s = sindy_library_adj(X, adj_matrix)

In [59]:
s.shape

torch.Size([310, 106, 10])

In [69]:
from DMD.sindy import sindy_fit2

# s = rearrange(s, 'n t f -> (t n) f', n=s.shape[0], t=s.shape[1], f=10)

x = rearrange(s[:,1:,:], 'n t f -> (t n) f')
y = rearrange(s[:,:-1,:], 'n t f -> (t n) f')

K = sindy_fit2(x, y, 1.0)

In [70]:
K.shape

(10, 10)